In [10]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [11]:
#use temporarily until tf.train bug is fixed 
#violet's mini batch code
class BatchSampler(object):
    '''
    A (very) simple wrapper to randomly sample batches without replacement.
    '''
    
    def __init__(self, data, targets, batch_size):
        self.num_points = data.shape[0]
        self.features = data.shape[1]
        self.data = data
        self.targets = targets
        self.batch_size = batch_size
        self.indices = np.arange(self.num_points)

    def random_batch_indices(self, m=None):
        if m is None:
            indices = np.random.choice(self.indices, self.batch_size, replace=False)
        else:
            indices = np.random.choice(self.indices, m, replace=False)
        return indices 

    def get_batch(self, m=None):
        '''
        Get a random batch without replacement from the dataset.
        If m is given the batch will be of size m. 
        Otherwise will default to the class initialized value.
        '''
        indices = self.random_batch_indices(m)
        X_batch = np.take(self.data, indices, 0)
        y_batch = self.targets[indices]
        return X_batch, y_batch

In [12]:
with np.load("notMNIST.npz") as data:
    Data, Target = data ["images"], data["labels"]
    np.random.seed(521)
    randIndx = np.arange(len(Data))
    np.random.shuffle(randIndx)
    Data = Data[randIndx]/255.
    Target = Target[randIndx]
    trainData, trainTarget = Data[:15000], Target[:15000]
    validData, validTarget = Data[15000:16000], Target[15000:16000]
    testData, testTarget = Data[16000:], Target[16000:]

In [13]:
trainData = np.reshape(trainData, [trainData.shape[0], 28*28])
validData = np.reshape(validData, [validData.shape[0], 28*28])
testData = np.reshape(testData, [testData.shape[0], 28*28])

In [15]:
#hyperparameters

learning_rates = [0.005, 0.001, 0.0001]
# learning_rates = [0.001]
mini_batch = 500
reg_coeff = 0.01
train_iter = 5000
train_loss = []

for learning_rate in learning_rates:
    
    tf.reset_default_graph()
    train_loss_curr = []
    
    x = tf.placeholder(tf.float32, shape = [None, 784])
    y = tf.placeholder(tf.float32, shape = [None, 10])

    
    w = tf.Variable(tf.random_normal([784, 10], mean = 0, stddev = 0.1))
    
    b = tf.cast(tf.Variable(0), dtype = tf.float32)
    y_hat = tf.add(tf.matmul(x, w), b)
    yhat = tf.sigmoid(y_hat)

    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = y, logits = yhat))
    
    regul = tf.multiply(reg_coeff, tf.reduce_mean(tf.square(w)))
    regul = tf.multiply(0.50, regul)
    cost = tf.add(cross_entropy, regul)
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)
    
    init = tf.global_variables_initializer()
    sess = tf.Session()
    sess.run(init)
    
    for epoch in range(train_iter):
#         x_batch, y_batch = tf.train.batch([trainData, trainTarget], mini_batch)
        
        trainBatchSampler = BatchSampler(trainData, trainTarget, mini_batch)
        x_batch, y_batch = trainBatchSampler.get_batch()
        
        sess.run(optimizer, feed_dict = {x: x_batch, y:y_batch})
        w_curr, b_curr, c = sess.run([w, b, cost], feed_dict = {x: x_batch, y:y_batch})
        train_loss_curr.append(c)
        
        if epoch%500 is 0:
            print("epoch:", epoch/500, " -> curr loss:", c)
            
    train_loss.append(train_loss_curr)
    
print("done")

ValueError: Cannot feed value of shape (500,) for Tensor 'Placeholder_1:0', which has shape '(?, 10)'